# NoteBook To create all sorts of DataSets

Save all new datasets in `data/new datasets/`

All datasets should have their own subFolder with files 

`x_train` columns: All variable names with index 0-XXX in numbers (no arrays)

`y_train` column: y column of 0/1 boolean values

`x_pred` same as x_train

### IMPORTS

In [149]:
using CSV, DataFrames, Statistics, Dates, Gadfly, LinearAlgebra, Distributions, Random, ScikitLearn, GLM

# Fonctions Globales

In [150]:
"""
    splitdataframe(df::DataFrame, p::Real)

Partitionne en un ensemble d'entraînement et un ensemble de validation un DataFrame.

### Arguments
- `df::DataFrame` : Un DataFrame
- `p::Real` : La proportion (entre 0 et 1) de données dans l'ensemble d'entraînement.

### Détails

La fonction renvoie deux DataFrames, un pour l'ensemble d'entraînement et l'autre pour l'ensemble de validation.

### Exemple

\```
 julia> splitdataframe(df, p.7)
\```

"""
function splitdataframe(df::DataFrame, p::Real)
   @assert 0 <= p <= 1 
    
    n = size(df,1)
    
    ind = shuffle(1:n)
    
    threshold = Int64(round(n*p))
    
    indTrain = sort(ind[1:threshold])
    
    indTest = setdiff(1:n,indTrain)
    
    dfTrain = df[indTrain,:]
    dfTest = df[indTest,:]
    
    return dfTrain, dfTest
    
end

splitdataframe

# Chargement des données et nettoyage préliminaire

## Chargement des surverses

In [151]:
data = CSV.read("./data/surverses.csv", missingstring="-99999")
first(data,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String⍰
1,0642-01D,2013-05-01,0,missing
2,0642-01D,2013-05-02,0,missing
3,0642-01D,2013-05-03,0,missing
4,0642-01D,2013-05-04,0,missing
5,0642-01D,2013-05-05,0,missing


## Nettoyage des données sur les surverses

#### Extraction des surverses pour les mois de mai à octobre inclusivement

In [152]:
data = filter(row -> month(row.DATE) > 4, data) 
data = filter(row -> month(row.DATE) < 11, data) 
first(data,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String⍰
1,0642-01D,2013-05-01,0,missing
2,0642-01D,2013-05-02,0,missing
3,0642-01D,2013-05-03,0,missing
4,0642-01D,2013-05-04,0,missing
5,0642-01D,2013-05-05,0,missing


#### Remplacement des valeurs *missing* dans la colonne :RAISON par "Inconnue"

In [153]:
raison = coalesce.(data[:,:RAISON],"Inconnue")
data[!,:RAISON] = raison
first(data,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String
1,0642-01D,2013-05-01,0,Inconnue
2,0642-01D,2013-05-02,0,Inconnue
3,0642-01D,2013-05-03,0,Inconnue
4,0642-01D,2013-05-04,0,Inconnue
5,0642-01D,2013-05-05,0,Inconnue


#### Exlusion des surverses coccasionnées par d'autres facteurs que les précipitations liquides

Ces facteurs correspondent à : 
- la fonte de neige (F), 
- les travaux planifiés et entretien (TPL)
- urgence (U)
- autre (AUT)

In [154]:
data = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], data) 
select!(data, [:NO_OUVRAGE, :DATE, :SURVERSE])
first(data,5)

,NO_OUVRAGE,DATE,SURVERSE
,String,Date,Int64⍰
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0


#### Exclusion des lignes où :SURVERSE est manquante

In [155]:
surverse_df = dropmissing(data, disallowmissing=true)
rename!(surverse_df, :DATE=>:date)
first(surverse_df,5)

,NO_OUVRAGE,date,SURVERSE
,String,Date,Int64
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0


## Chargement des précipitations

In [156]:
data = CSV.read("data/precipitations.csv",missingstring="-99999")
rename!(data, Symbol("St-Hubert")=>:StHubert)
first(data,5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-01-01,0,0,0,0,0,missing
2,2013-01-01,1,0,0,0,0,missing
3,2013-01-01,2,0,0,0,0,missing
4,2013-01-01,3,0,0,0,0,missing
5,2013-01-01,4,0,0,0,0,missing


## Nettoyage des données sur les précipitations

#### Extraction des précipitations des mois de mai à octobre inclusivement

In [157]:
data = filter(row -> month(row.date) > 4, data) 
data = filter(row -> month(row.date) < 11, data) 
first(data,5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-05-01,0,0,0,0,0,missing
2,2013-05-01,1,0,0,0,0,missing
3,2013-05-01,2,0,0,0,0,missing
4,2013-05-01,3,0,0,0,0,missing
5,2013-05-01,4,0,0,0,0,missing


 ### Remplissage des données manquantes
Nous allons tenter de remplir les données manquantes par des moyennes de précipitations lorsque les données sont inconnues pour 2 stations ou plus afin d'avoir plus de données pour créer nos modèles

# Techniques pour remplir les données manquantes

- utiliser regression ridge pour remplir les ligne ou il y a une valeur manquante 
- remplir les lignes avec plusieurs valeurs manquantes avec la moyenne des valeurs présentes

## Fonction Ridge pour trouver les valueurs manquantes

In [158]:
# Fonction pour faire une regression ridge
# Ressort le beta, m, et s
function ridge(datas::DataFrame, station::Symbol)
       
    Train, Test = splitdataframe(datas, .75);
    # Prétraitement des données
    # Les variables avec les tildes correspondent à l'échantillon de test

    X = convert(Matrix{Int64},Train[:,Not(station)])
    m = mean(X, dims=1)
    s = std(X, dims=1)
    m[2] = 0
    s[2] = 1
    X = (X .- m) ./ s

    X̃ = convert(Matrix{Int64},Test[:,Not(station)])
    X̃ = (X̃ .- m) ./ s

    y = convert(Vector{Int64}, Train[:,station])
    m = mean(y)
    s = std(y)
    y = (y .- m) ./s

    ỹ = convert(Vector{Int64}, Test[:,station])
    ỹ = (ỹ .- m) ./s;

    #On calcule ensuite le RMSE pour chacun des valeurs de lambda
    RMSEs = DataFrame(λ=Float64[], RMSE=Float64[])

    for λ in 0:1:10000
   
        β̂ = (X'X + λ*I)\X'y
    
        ŷ = X̃*β̂
    
        ẽ = ỹ - ŷ
    
        RMSE = sqrt(dot(ẽ,ẽ)/length(ẽ))
    
        push!(RMSEs, [λ, RMSE])
    
    end
    
    # On trouve ensuite la valeure de lambda qui minimise le RMSE
    _, ind = findmin(RMSEs[:,:RMSE])

    λ̂ = RMSEs[ind,:λ]
    
    β̂ = (X'X + λ̂*I)\X'y
    
    #TODO validate model and print value of validator R² ajuste
    
    #On peut alors calculer les y avec les betas trouver et l'echantillon de test
    ŷ = X̃ * β̂
    ŷ = round.((ŷ .* s) .+ m)
    
    # Calcul du R² ajusté

    p = 4          # nombre de variables explicatives
    n = length(ỹ)  # taille de l'échantillon

    ỹ = (ỹ .* s) .+ m
    ȳ = mean(ỹ)
    e = ỹ - ŷ

    SST = sum( (ỹ[i] - ȳ)^2 for i=1:n )  # variabilité totale
    SSE = sum( e.^2 )                    # variabilité résiduelle

    R2aj =  1 - SSE/SST * (n-1)/(n-p)
    
    println("Le R² ajuste du modele trouve pour la station de $(station) est $(R2aj)")
    
    return β̂
end

ridge (generic function with 1 method)

In [160]:
# utilisations des données connues pour bâtire les modèles de prédiction Ridge
full_data = dropmissing(data, disallowmissing=true)
full_data = full_data[:,Not(:date)][:, Not(:heure)]
size(full_data)

(20001, 5)

In [161]:
# Liste des betas par station manquante (prédite)
betas = DataFrame(station = Symbol[], β = Array{Float64}[])
for name in names(full_data)
    β̂ = ridge(full_data, name)
    push!(betas, [name, β̂])
end

Le R² ajuste du modele trouve pour la station de McTavish est 0.6818964239458101
Le R² ajuste du modele trouve pour la station de Bellevue est 0.44965617430197047
Le R² ajuste du modele trouve pour la station de Assomption est 0.2259669596460928
Le R² ajuste du modele trouve pour la station de Trudeau est 0.6124946856258109
Le R² ajuste du modele trouve pour la station de StHubert est 0.4706529979050087


## Remplissage des données en effectuant la moyenne des données présentes si plus de 1 est manquante, sinon utilisation du modèle Ridge approprié à la station manquante

In [162]:
include("datasets/countMissing.jl")
include("datasets/meanLine.jl")
include("datasets/replaceMissing.jl")
precipitation_df = data[:,Not(:date)][:,Not(:heure)]
for row in eachrow(precipitation_df)
    nbMissing, ind = countMissing(row)
    if(nbMissing == 1)
        row[ind] = round((convert(Vector{Float64},row[Not(ind)])'*betas[:, :β][ind]))
    end
    # remplacer les lignes qui ont de 2 a 4 missing
    if(nbMissing<5 && nbMissing>1)
        replaceMissing(row,round(meanLine(row)))
    end
end
precipitation_df.heure = data[:,:heure]
precipitation_df.date = data[:,:date]
precipitation_df = dropmissing(precipitation_df) # drop all missing
CSV.write("data/new_datasets/precipitation_filed_mean_per_hour.csv",precipitation_df)
first(precipitation_df,10)

,McTavish,Bellevue,Assomption,Trudeau,StHubert,heure,date
,Int64,Int64,Int64,Int64,Int64,Int64,Date
1,0,0,0,0,0,0,2013-05-01
2,0,0,0,0,0,1,2013-05-01
3,0,0,0,0,0,2,2013-05-01
4,0,0,0,0,0,3,2013-05-01
5,0,0,0,0,0,4,2013-05-01
6,0,0,0,0,0,5,2013-05-01
7,0,0,0,0,0,6,2013-05-01
8,0,0,0,0,0,7,2013-05-01
9,0,0,0,0,0,8,2013-05-01


## Nouvelle variable explicative : somme des précipitations par jour pour les stations de précipitations
- Nous pensons que cela pourrait bien expliquer des surverses

In [163]:
precipitation_daily_sum = by(precipitation_df, :date,  McTavish = :McTavish=>sum, Bellevue = :Bellevue=>sum, 
   Assomption = :Assomption=>sum, Trudeau = :Trudeau=>sum, StHubert = :StHubert=>sum)
last(precipitation_daily_sum ,10)

,date,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64,Int64,Int64,Int64
1,2019-10-21,0,0,0,0,0
2,2019-10-22,104,109,109,129,92
3,2019-10-23,152,138,138,114,149
4,2019-10-24,0,0,0,0,0
5,2019-10-25,0,0,0,0,0
6,2019-10-26,0,0,0,0,0
7,2019-10-27,442,378,378,352,341
8,2019-10-28,7,6,6,2,6
9,2019-10-29,0,1,1,0,2


### Extraire l'année 2019 pour les données à prédire

In [164]:
precipitation_daily_sum_train = filter(row -> Year(row[:date]) != Year(2019), precipitation_daily_sum)
precipitation_daily_sum_pred  = filter(row -> Year(row[:date]) == Year(2019), precipitation_daily_sum)

,date,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64,Int64,Int64,Int64
1,2019-05-01,79,52,58,47,68
2,2019-05-02,26,19,15,13,17
3,2019-05-03,34,27,34,31,44
4,2019-05-04,2,0,0,2,0
5,2019-05-05,0,0,0,0,0
6,2019-05-06,0,0,0,0,0
7,2019-05-07,3,0,0,0,0
8,2019-05-08,0,0,0,0,0
9,2019-05-09,89,67,77,86,67


#### Envoyer vers fichier CSV

In [165]:
filter!(row -> row.date in surverse_df[!, :date], precipitation_daily_sum_train)
filter!(row -> row.date in precipitation_daily_sum_train[!, :date], surverse_df)

,NO_OUVRAGE,date,SURVERSE
,String,Date,Int64
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0
6,0642-01D,2013-05-06,0
7,0642-01D,2013-05-07,0
8,0642-01D,2013-05-08,0
9,0642-01D,2013-05-09,0


In [166]:
CSV.write("data/new_datasets/precipitation_daily_sum/x_train.csv", precipitation_daily_sum_train)
CSV.write("data/new_datasets/precipitation_daily_sum/x_pred.csv", precipitation_daily_sum_pred)

"data/new_datasets/precipitation_daily_sum/x_pred.csv"

## Nouvelle variable explicative : Maximum des précipitations par jour pour les stations de précipitations
- Nous pensons que cela pourrait bien expliquer des surverses

#### Extraction du taux horaire journalier maximum des précipitations pour chacune des stations météorologiques

In [167]:
precipitation_daily_max = by(precipitation_df, :date,  McTavish = :McTavish=>maximum, Bellevue = :Bellevue=>maximum, 
   Assomption = :Assomption=>maximum, Trudeau = :Trudeau=>maximum, StHubert = :StHubert=>maximum)
first(precipitation_daily_max,10)

,date,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64,Int64,Int64,Int64
1,2013-05-01,0,0,0,0,0
2,2013-05-02,0,0,0,0,0
3,2013-05-03,0,0,0,0,0
4,2013-05-04,0,0,0,0,0
5,2013-05-05,0,0,0,0,0
6,2013-05-06,0,0,0,0,0
7,2013-05-07,0,0,0,0,0
8,2013-05-08,0,0,0,0,0
9,2013-05-09,10,0,19,0,7


### Extraire l'année 2019 pour les données à prédire


In [168]:
precipitation_daily_max_train = filter(row -> Year(row[:date]) != Year(2019), precipitation_daily_max)
precipitation_daily_max_pred  = filter(row -> Year(row[:date]) == Year(2019), precipitation_daily_max)

,date,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64,Int64,Int64,Int64
1,2019-05-01,24,24,15,21,22
2,2019-05-02,12,7,7,8,10
3,2019-05-03,12,11,14,14,15
4,2019-05-04,2,0,0,2,0
5,2019-05-05,0,0,0,0,0
6,2019-05-06,0,0,0,0,0
7,2019-05-07,3,0,0,0,0
8,2019-05-08,0,0,0,0,0
9,2019-05-09,45,26,55,43,32


#### Envoyer vers fichier CSV
Valider que les dates correspondent pour x et y

In [169]:
filter!(row -> row.date in surverse_df[!, :date], precipitation_daily_max_train)
filter!(row -> row.date in precipitation_daily_max_train[!, :date], surverse_df)

,NO_OUVRAGE,date,SURVERSE
,String,Date,Int64
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0
6,0642-01D,2013-05-06,0
7,0642-01D,2013-05-07,0
8,0642-01D,2013-05-08,0
9,0642-01D,2013-05-09,0


In [170]:
CSV.write("./data/new_datasets/precipitation_daily_max/x_train.csv", precipitation_daily_max_train)
CSV.write("./data/new_datasets/precipitation_daily_max/x_pred.csv", precipitation_daily_max_pred)

"./data/new_datasets/precipitation_daily_max/x_pred.csv"

In [171]:
CSV.write("./data/new_datasets/surverse_list.csv", surverse_df)

"./data/new_datasets/surverse_list.csv"

## Nouvelle variable explicative : somme plus importante de précipitations par plages horaires sur une journée 
- Nous avons extrait cette varibale explicative avec l'idéé en tête que quelsques heures consécutives avec beaucoup de précipitations pourraient causer des surverses plus que des précipitations modérées réparties sur une journée complète

In [186]:
filledPrec  = CSV.read("data/new_datasets/precipitation_filed_mean_per_hour.csv", missingstring="-99999")
first(filledPrec,5)

,McTavish,Bellevue,Assomption,Trudeau,StHubert,heure,date
,Int64,Int64,Int64,Int64,Int64,Int64,Date
1,0,0,0,0,0,0,2013-05-01
2,0,0,0,0,0,1,2013-05-01
3,0,0,0,0,0,2,2013-05-01
4,0,0,0,0,0,3,2013-05-01
5,0,0,0,0,0,4,2013-05-01


#### Fonctions pour séparer les journées en plages horaires de taille "window" et effectuer la somme des précipiations sur chaque plage

In [187]:
function dayPrecipitationSplit(window,precipitations)
    n = size(precipitations,1)
    newDf = DataFrame(McTavish = Int64[], Bellevue = Int64[], Assomption = Int64[], Trudeau  = Int64[],
                        StHubert = Int64[], heureDebut = Int64[], heureFin = Int64[], date = Date[])

    hourgroups = 24/window
    for day in groupby(filledPrec, :date)
        start = 1
        finish = window
        date = day[1,:date]
        if(size(day, 1)==24)
            for i=1:hourgroups
                mcTavish, Bellevue, Assomption, Trudeau, StHubert, heureDebut, heureFin = 0,0,0,0,0,0,0
                for j=start:finish
                    mcTavish += day[j,:McTavish]
                    Assomption += day[j,:Assomption]
                    Bellevue += day[j,:Bellevue]
                    Trudeau += day[j,:Trudeau]
                    StHubert += day[j,:StHubert]
                    if j == start
                        heureDebut = day[j,:heure]
                    elseif j == finish
                        heureFin = day[j,:heure]
                    end
                end
                start += window
                finish += window
                push!(newDf,[mcTavish,Bellevue,Assomption,Trudeau,StHubert,heureDebut,heureFin,date])
            end
        end
    end
    return newDf
end

dayPrecipitationSplit (generic function with 1 method)

#### Générations des sommes de plages horaires pour des divisions de journées de 2,3,4,6,8 et 12 heures

In [189]:
hourSplit2 = dayPrecipitationSplit(2,filledPrec)
hourSplit3 = dayPrecipitationSplit(3,filledPrec)
hourSplit4 = dayPrecipitationSplit(4,filledPrec)
hourSplit6 = dayPrecipitationSplit(6,filledPrec)
hourSplit8 = dayPrecipitationSplit(8,filledPrec)
hourSplit12 = dayPrecipitationSplit(12,filledPrec)
first(hourSplit12,5)

,McTavish,Bellevue,Assomption,Trudeau,StHubert,heureDebut,heureFin,date
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Date
1,0,0,0,0,0,0,11,2013-05-01
2,0,0,0,0,0,12,23,2013-05-01
3,0,0,0,0,0,0,11,2013-05-02
4,0,0,0,0,0,12,23,2013-05-02
5,0,0,0,0,0,0,11,2013-05-03


#### Fonction pour extraire la plage horaire avec la somme des précipitations la plus grande pour chaque jour

In [190]:
function maxPrecByDay(Prec)
    n = size(Prec,1)
    newDf = DataFrame(McTavish = Int64[], Bellevue = Int64[], Assomption = Int64[], Trudeau  = Int64[],
                        StHubert = Int64[], date = Date[])

    for day in groupby(Prec, :date)
        mcTavish = maximum(day[:,:McTavish])
        Assomption = maximum(day[:,:Assomption])
        Bellevue = maximum(day[:,:Bellevue])
        Trudeau = maximum(day[:,:Trudeau])
        StHubert = maximum(day[:,:StHubert])
        date = day[1,:date]
        push!(newDf,[mcTavish,Bellevue,Assomption,Trudeau,StHubert,date])
    end
    return newDf
end

maxPrecByDay (generic function with 1 method)

#### Datasets contenant la nouvelle variables explicative pour les 5 stations pour les journées divisée par sections de 2,3,4,6,8 et 12 heures 

In [192]:
maxSum2hours = maxPrecByDay(hourSplit2)
maxSum3hours = maxPrecByDay(hourSplit3)
maxSum4hours = maxPrecByDay(hourSplit4)
maxSum6hours = maxPrecByDay(hourSplit6)
maxSum8hours = maxPrecByDay(hourSplit8)
maxSum12hours = maxPrecByDay(hourSplit12)
first(maxSum12hours,5)

,McTavish,Bellevue,Assomption,Trudeau,StHubert,date
,Int64,Int64,Int64,Int64,Int64,Date
1,0,0,0,0,0,2013-05-01
2,0,0,0,0,0,2013-05-02
3,0,0,0,0,0,2013-05-03
4,0,0,0,0,0,2013-05-04
5,0,0,0,0,0,2013-05-05


#### Écriture des dataframes ver des fichiers CSV

In [178]:
CSV.write("./data/new_datasets/max_precipitation_day_split/maxPrecBy2hours.csv",maxSum2hours)
CSV.write("./data/new_datasets/max_precipitation_day_split/maxPrecBy3hours.csv",maxSum3hours)
CSV.write("./data/new_datasets/max_precipitation_day_split/maxPrecBy4hours.csv",maxSum4hours)
CSV.write("./data/new_datasets/max_precipitation_day_split/maxPrecBy6hours.csv",maxSum6hours)
CSV.write("./data/new_datasets/max_precipitation_day_split/maxPrecBy8hours.csv",maxSum8hours)
CSV.write("./data/new_datasets/max_precipitation_day_split/maxPrecBy12hours.csv",maxSum12hours)

"./data/new_datasets/max_precipitation_day_split/maxPrecBy12hours.csv"

## Nouvelle variable explicative : Somme de la journee + les deux dernieres heures du jour précédent
- Nous avons décidé d'ajouter les deux heures de la journée précedente, car nous croyons que les dernières heures de cette journée peuvent effet impoartant sur la journée qui suit et pas sur celle dont elles font parite

In [194]:
filledPrec  = CSV.read("data/new_datasets/precipitation_filed_mean_per_hour.csv", missingstring="-99999")
first(filledPrec,5)

,McTavish,Bellevue,Assomption,Trudeau,StHubert,heure,date
,Int64,Int64,Int64,Int64,Int64,Int64,Date
1,0,0,0,0,0,0,2013-05-01
2,0,0,0,0,0,1,2013-05-01
3,0,0,0,0,0,2,2013-05-01
4,0,0,0,0,0,3,2013-05-01
5,0,0,0,0,0,4,2013-05-01


#### Somme et ajout des 2 dernières heures de la journée précédente pour les précipitations

In [196]:
for i=1:size(precipitation_daily_sum, 1)
    ind = findfirst(filledPrec[:,:date] .== precipitation_daily_sum[i,:date])
    for h=1:2
        for key in names(precipitation_daily_sum[:, Not(:date)])
            if ind-h > 0
                precipitation_daily_sum[i, key] += filledPrec[ind-h, key]
            end
        end
    end
end
first(precipitation_daily_sum, 5)

,date,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64,Int64,Int64,Int64
1,2013-05-01,0,0,0,0,0
2,2013-05-02,0,0,0,0,0
3,2013-05-03,0,0,0,0,0
4,2013-05-04,0,0,0,0,0
5,2013-05-05,0,0,0,0,0


#### Exportation vers CSV

In [182]:
CSV.write("./data/new_datasets/sum_day_last_2.csv",precipitation_daily_sum)

"./data/new_datasets/sum_day_last_2.csv"

##### Dataset avec toutes les surverses des autres ouvrages ayant assez de données par date
- Nous n'avons finalement pas utilisé ces données, car il y avait une quatité très importante de données manquantes et nous n'avions pas accès aux données pour 2019 afin d'effectuer les préditions

In [183]:
surversesVoisines = DataFrame()
surversesVoisines[:,:date] = maxSum2hours[:,:date]
ouvrages = groupby(surverse_df, :NO_OUVRAGE)
for subdf in ouvrages
            length = (size(subdf, 1))
            if (length > 1000)
                subdf = rename(subdf,:SURVERSE => subdf[1,:NO_OUVRAGE])
                dateAndSurverse = subdf[:,Not(:NO_OUVRAGE)]
                surversesVoisines = join(surversesVoisines, dateAndSurverse, on = :date, kind = :left)    
            end
       end
first(surversesVoisines,5)
CSV.write("./data/new_datasets/surversesVoisines",surversesVoisines)


"./data/new_datasets/surversesVoisines"